<p> Run a simulation of filling in a network based on the 7 de Septiembre neighborhood layout in Araijan </p>
<p>**(.inp and .config files already written)**</p>
<p> Below you see how to:</p>
<ul>
<li>Set boundary and initial conditions </li>
<li>Run a simulation until time T= 1200 s</li>
<li>plot time series of pressure head at different points in a single pipe</li>
<li>plot time series of pressure head at sample points in various pipes</li>
<li>plot space dependence of pressure head at a certain time</li>
<li>look at velocities</li>
<li>plot network layout</li>
</ul>

In [1]:
from __future__ import division
import sys
sys.path.append("/home/xin/pipes")  
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
#%pylab inline
%pylab notebook
from writeit import *
import pickle
import time
from datetime import datetime, timedelta

Populating the interactive namespace from numpy and matplotlib


In [2]:
fi = "/home/xin/pipes/indata/Xindata/Saket.inp"    
fc = "/home/xin/pipes/indata/Xindata/Saket.config" 
mtype = 1                          
n1 = PyNetwork(fi,fc,mtype)

In [3]:
########################## Set IC for all pipes #################################
pipe_ini = PyPipe_ps(n1.Ns[0],n1.Ds[0],n1.Ls[0],n1.M,n1.a[0])

# IC of the pipe (dry)
'''
 conn: np.ndarray
       Nedgesx2 array of ints. Row i = [start node, end node] for pipe i.
'''
for pipe in xrange(len(n1.conn)):
#     A_crt = n1.Ds[pipe]**2*np.pi/4.*1e-2
#     print pipe_ini.HofA(A_crt, True)
    L = n1.Ls[pipe]
    N = n1.Ns[pipe]
    D = n1.Ds[pipe]
    pipe_obj = PyPipe_ps(N,D,L,n1.M,n1.a[0])
#     A_crt = pipe_obj.AofH(1e-4, False)
    A_crt = pipe_obj.AofH(0.2, False)
    Q_crt = 1e-2*A_crt
    Ni = n1.Ns[pipe]
    n1.setIC(pipe, A_crt*np.ones(Ni), Q_crt*np.ones(Ni))
print 'IC completed'

IC completed


In [4]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib
########################### Get pressure boundary series ##########################
df = pd.read_csv('/home/xin/pipes/indata/Xindata/MayCalibTransient.csv')
Time_str = df["Time"]
Hs = df["OHT(2)"]
JB = df["JB"]
KB = df["KB"]

####### input requriement, convert negative pressures to be 0
Hs[Hs < 0] = 0.

whole_picture_flag = False
if whole_picture_flag == True:
    start_index = 0 #299
    end_index = len(Time) 
else:
    start_index = 303 #299
    end_index = 603   #499

print n1.getTotalVolume()

177.662449076


/home/xin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
############### Set Parameters for loop ###########################
Hs_in = Hs[start_index:end_index].tolist()
T_total = n1.T*(len(Hs_in)-1)
###################################################################
M = n1.M
T = n1.T
# i = 0 means A, i=1 means Q
# j means the grid we are currently looking at, 1<=j<=N
# n means time step 
# N means the grid number in this certain pipe
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

# conn: np.ndarray
#       Nedgesx2 array of ints. Row i = [start node, end node] for pipe i.
pipe_number = len(n1.conn)
model_number = int(T_total/n1.T)
dt = n1.T/float(n1.M)


# # Store all the pressure head data at KB
# KB_numerical = []  
# # Store all the pressure head data at JB 
# JB_numerical = [] 
# Store the IC condition for each round of simulation
# ICs = { pipe: [ [A],[Q] ] }, therefore A = ICs[pipe][0], Q = ICs[pipe][1] 
ICs = {}
initial_flag = True

flux_file = "/home/xin/pipes/indata/Xindata/flux_more_revised.csv"
import os
try:
    os.remove(flux_file)
except:
    print "File not created yet"
        
flux_nodes =[39, 45, 46, 48, 49, 50, 51, 53, 54, 55, 57, 59, 60, 62, 63, 64, 65, 67, 68, 70, 71, 73, 77, 78,
            81, 82, 84, 86, 88, 90, 91, 92, 94, 95, 97, 98, 99]
end_Qs = {}
with open(flux_file, 'a') as f:
    for flux_node in flux_nodes:
        f.write("node %i,"%flux_node)
    f.write("\n")
    for k in xrange(model_number):
        A_in_start = pipe_ini.AofH(Hs_in[k], True) 
        A_in_end = pipe_ini.AofH(Hs_in[k+1], True)
        print "The current boundary is",Hs_in[k],"--",Hs_in[k+1]
        A_boun = np.linspace(A_in_start,A_in_end,n1.M+1)    
        n1.setbVal(0,A_boun)
        if k != 0:
            t1 = time.time()
            for pipe in xrange(pipe_number):
                n1.setIC(pipe, ICs[pipe][0], ICs[pipe][1])
            n1.runForwardProblem(dt)
            t2 = time.time()
        else:
            t1 = time.time()
            n1.runForwardProblem(dt)
            t2 = time.time()
        print 'Piece %d of %d, running time = %.1fs'%(k+1, model_number, t2-t1)
        try:
            os.remove("/home/xin/pipes/indata/Xindata/example_file.txt")
        except:
            print "Logging file not created yet"
        for pipe in xrange(pipe_number):
            qh = n1.qhist(pipe)
            N = n1.Ns[pipe]
            A = [qh[idx_t(0, ja, M, N)] for ja in xrange(1,N+1)]
            Q = [qh[idx_t(1, jq, M, N)] for jq in xrange(1,N+1)]
            ICs.update({pipe: [A, Q]})         
            end_node = pipe + 1
            print pipe, end_node
            if end_node in flux_nodes:
                end_Qs.update({end_node: [qh[idx_t(1, N, jq, N)] for jq in xrange(1,M+1)]}) 
#                 end_Qs.update({end_node: n1.pressureTimeSeries(pipe,N)})  
        end_Qs.keys().sort()
        print end_Qs.keys()
        flag = True
        for j_num in xrange(M):
            for i_num in end_Qs:
                if flag:
                    print i_num
                f.write("%.4f,"%end_Qs[i_num][j_num])
            flag = False
            f.write("\n")
        n1.reset()
        time.sleep(1)

The current boundary is 0.064729275 -- 0.113543643
Piece 1 of 299, running time = 1286.8s
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 60
60 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
[64, 65, 67, 68, 70, 71, 73, 77, 78, 81, 82, 84, 86, 88, 90, 91, 92, 94, 95, 97, 98, 99, 39, 45, 46, 48, 49, 50, 51, 53, 54, 55, 57, 59, 60, 62, 63]
64
65
67
68
70
71
73
77
78
81
82
84
86
88
90
91
92
94
95
97
98
99
39
45
46
48
49
50
51
53
54
55
57
59
60
62
63
The current boundary is 0.113543643 -- 0.10987382
Piece 2 of 299, running t

In [ ]:
############### Set Parameters for loop ###########################
Hs_in = Hs[start_index:end_index].tolist()
T_total = n1.T*(len(Hs_in)-1)
###################################################################
M = n1.M
T = n1.T
# i = 0 means A, i=1 means Q
# j means the grid we are currently looking at, 1<=j<=N
# n means time step 
# N means the grid number in this certain pipe
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

# conn: np.ndarray
#       Nedgesx2 array of ints. Row i = [start node, end node] for pipe i.
pipe_number = len(n1.conn)
model_number = int(T_total/n1.T)
dt = n1.T/float(n1.M)


# # Store all the pressure head data at KB
# KB_numerical = []  
# # Store all the pressure head data at JB 
# JB_numerical = [] 
# Store the IC condition for each round of simulation
# ICs = { pipe: [ [A],[Q] ] }, therefore A = ICs[pipe][0], Q = ICs[pipe][1] 
ICs = {}
initial_flag = True

num_logger_file = "/home/xin/pipes/indata/Xindata/Numerical_logger_tuned16.csv"
import os
try:
    os.remove(num_logger_file)
except:
    print "File not created yet"
        
with open(num_logger_file, 'a') as f:
    f.write("KB_numerical, JB_numerical \n")
    for k in xrange(model_number):
        A_in_start = pipe_ini.AofH(Hs_in[k], True) 
        A_in_end = pipe_ini.AofH(Hs_in[k+1], True)
        print "The current boundary is",Hs_in[k],"--",Hs_in[k+1]
        A_boun = np.linspace(A_in_start,A_in_end,n1.M+1)    
        n1.setbVal(0,A_boun)
        if k != 0:
            t1 = time.time()
            for pipe in xrange(pipe_number):
                n1.setIC(pipe, ICs[pipe][0], ICs[pipe][1])
            n1.runForwardProblem(dt)
            t2 = time.time()
        else:
            t1 = time.time()
            n1.runForwardProblem(dt)
            t2 = time.time()
        print 'Piece %d of %d, running time = %.1fs'%(k+1, model_number, t2-t1)
        try:
            os.remove("/home/xin/pipes/indata/Xindata/example_file.txt")
        except:
            print "Logging file not created yet"
        for pipe in xrange(pipe_number):
            qh = n1.qhist(pipe)
            N = n1.Ns[pipe]
            A = [qh[idx_t(0, ja, M, N)] for ja in xrange(1,N+1)]
            Q = [qh[idx_t(1, jq, M, N)] for jq in xrange(1,N+1)]
            ICs.update({pipe: [A, Q]})
        if not initial_flag: 
            # Cut the first number since the begining of this one and the end of last one are overlaped
            KB_numerical = n1.pressureTimeSeries(18,2)[1:]  #One grid before the last one 
            JB_numercial = n1.pressureTimeSeries(28,8)[1:]
        else:
            KB_numerical = n1.pressureTimeSeries(18,2)
            JB_numercial = n1.pressureTimeSeries(28,8)
            initial_flag = False
        for i_num in xrange(len(KB_numerical)):
            f.write("%.4f, %.4f \n"%(KB_numerical[i_num],JB_numercial[i_num]))
        n1.reset()
        time.sleep(1)

In [ ]:
### Read File and Plot
#### plot the three series of data time-varied figure
def right_legend(ax):
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.88))
start_index = 303 #299
end_index = 333   #499
num_logger_file = "/home/xin/pipes/indata/Xindata/Numerical_logger_tuned8.csv"
FMT = "%m/%d/%Y %H:%M"
Time = [datetime.strptime(i,FMT) for i in Time_str]
dates = matplotlib.dates.date2num(Time)


df2 = pd.read_csv(num_logger_file)
KB_num = df2["KB_numerical"]
JB_num = df2[" JB_numerical "]
x_num = np.linspace(dates[start_index],dates[end_index], len(KB_num))

fig = plt.figure(figsize = (12,5))
ax = fig.add_subplot(1, 1, 1)
ax.plot_date(dates[start_index:end_index], Hs[start_index:end_index],c = 'blue', marker = "o",ms = 5, label = "Boundary")
ax.plot_date(dates[start_index:end_index], KB[start_index:end_index],c ='red',marker = '<',ms = 5, label = "KB")
ax.plot_date(dates[start_index:end_index], JB[start_index:end_index],c ='yellow', marker = '*',ms = 6, label = "JB")
ax.plot_date(x_num, KB_num, 'r-', label = "KB_numerical")
ax.plot_date(x_num, JB_num,'y-',label = "JB_numerical")
plt.xticks(rotation=0)
ax.set_xlabel("Time")
ax.set_ylabel("Pressure(m)")
right_legend(ax)



In [ ]:
############ Inflpow Boundary Values (m^3/s) #########################
v = 1.5   #inflow boundary velocity (m/s), used to create the Q0 vector
######################################################################
M = n1.M  
T = n1.T  
q0 = v*n1.Ds[0]**2*np.pi/4.
Q_inflow = q0*np.ones(M+1)          
n1.setbVal(0,Q_inflow)   #boundary condition for junction 0 (at the left end of pipe 0)
print 'Inflow boundary finished'


'''
The Optimal orfice here should be set as follows:
1) Fully open until pressurized (not ploted in field data)
2) Gradually decrease the opening to 0.1 & 0.05 (reach the first stage)
3) Keep there for 4 minutes
4) Gradually decrease the opening to 0.0729 & 0.0176 (or a higher number consider 15m head)
'''

# M1 = int((M+1)/3)
# Open1 = np.ones(M+1)
# Open1[:M1+1] = [0.12 - 0.02*i/M1 for i in xrange(len(Open1[:M1+1]))]
# Open1[M1+1:2*M1+2] = [0.1 for i in Open1[M1+1:2*M1+2]]
# Open1[2*M1+2:] = [0.1 - (0.1-0.0729)*i/M1 for i in xrange(len(Open1[2*M1+2:]))]
# n1.setbVal(1,Open1)

# Open2 = np.ones(M+1)
# Open2[:M1+1] = [0.05-0.00*i/M1 for i in xrange(len(Open1[:M1+1]))]
# Open2[M1+1:2*M1+2] = [0.05 for i in Open2[M1+1:2*M1+2]]
# Open2[2*M1+2:] = [0.05 - (0.05-0.0176)*i/M1 for i in xrange(len(Open2[2*M1+2:]))]
# n1.setbVal(2,Open2)

Opening = 0.015*np.ones(M+1)
for orifice in xrange(1, 10):
    n1.setbVal(orifice,Opening)


########## Get some information about the pipe pressure head when full #######################
a = n1.a[0]
for i in xrange(len(n1.Ns)):
    p1 = PyPipe_ps(n1.Ns[i], n1.Ds[i],n1.Ls[i], M, a)
    Ap = n1.Ds[i]**2/4*np.pi  # For the first pipe, get the pipe cross-section area
    g = 9.81
    p_head = p1.Eta(Ap,True)/(g*Ap)
    slot_w = g*Ap/a**2
    print "For pipe %d, when it is full, its pressure head is %.4f m, slot width is %.3e"%(i,p_head, slot_w)
print 'Pipe info got '
   
'''   
########## Set Initial Condition #######################################
for i in xrange(0,len(n1.Ns)):
    A00 = 1e-4*np.ones(n1.Ns[i])
    Q00 = 5e-5*np.ones(n1.Ns[i])
    n1.setIC(i,A00,Q00)
print 'IC setup finished'
''' 

In [ ]:
######## Plot the change of orifice setting
# plt.figure()
# Mx = arange(M+1)*T/M/60
# plt.plot(Mx, Open1, label = 'Orifice 1, Main Pipe')
# plt.plot(Mx, Open2, label = 'Orifice 2, Branch Pipe')
# plt.xlabel('minutes')
# plt.ylabel('m')
# plt.title("Openings of Orificies")
# plt.legend()
# plt.show()

In [ ]:
######  Only For Check, do not need to run #############
'''
A0 = np.zeros(M+1)
n1.setbVal(0,A0)
n1.setbVal(1,A0)
n1.setbVal(2,A0)
'''
# M = n1.M
# a = n1.a[0]
# p0 = PyPipe_ps(n1.Ns[0], n1.Ds[0],n1.Ls[0], M, a)
# p6 = PyPipe_ps(n1.Ns[6], n1.Ds[6],n1.Ls[6], M, a)
# A0 = p0.AofH(0.8, True)
# A6 = p6.AofH(0.625, True)
# print p0.HofA(0.012826, False)
# print p6.HofA(0.009633, False)
#print p0.pbar(A0, False), p6.pbar(A6, False) , p0.HofA(0.290475, True), p0.AofH(0.0729*0.83, True)



In [ ]:
########### Plot model layout ##############################
"""
(xs,ys,conns,ls) = getBasicConnectivity(fi)
Np= shape(conns) [0]
%pylab notebook
plotNetworkLayout (xs, ys, conns, ls, Np) 
"""
########### Print initial data about the model ############# (Not necessary, but can check)
#n1.showLayout()
#n1.showCurrentData()

In [ ]:
#Run the simulation!
# print M
# dt = T/float(M)#time step
# V0 = n1.getTotalVolume()
# t1 = time.time()
# n1.runForwardProblem(dt)
# t2 = time.time()
# print 'Model run finished with time = %ds'%(t2-t1)

############### Set Parameters for loop ###########################
T_total = n1.T*2
###################################################################

# i = 0 means A, i=1 means Q
# j means the grid we are currently looking at, 1<=j<=N
# n means time step 
# N means the grid number in this certain pipe
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

# conn: np.ndarray
#       Nedgesx2 array of ints. Row i = [start node, end node] for pipe i.
pipe_number = len(n1.conn)
model_number = int(T_total/T)
dt = T/float(M)

# Store all the pressure head data before T
P_beforeT = [] 
# Store all the pressure head data after T
P_afterT = []  
# Store the IC condition for each round of simulation
# ICs = { pipe: [ [A],[Q] ] }, therefore A = ICs[pipe][0], Q = ICs[pipe][1] 
ICs = {}



for k in xrange(model_number):
    n1.setbVal(0,Q_inflow) 
    for orifice in xrange(1, 10):
        n1.setbVal(orifice,Opening)
#     qh0 = n1.qhist(2)
#     print qh0[100:120]
    if k != 0:
        #Open1 = 0.0729*np.ones(M+1)
        #n1.setbVal(1,Open1)
        #Open2 = 0.0176*np.ones(M+1)
        #n1.setbVal(2,Open2)
        t1 = time.time()
        for pipe in xrange(pipe_number):
            n1.setIC(pipe, ICs[pipe][0], ICs[pipe][1])
        n1.runForwardProblem(dt)
        t2 = time.time()
    else:
        t1 = time.time()
#         for pipe in xrange(pipe_number):
#             edge = n1.conn[pipe]
#             SetPipeIC(n1, pipe, bench_H, elev[edge[0]], elev[edge[1]])
        n1.runForwardProblem(dt)
        t2 = time.time()
    print 'Piece %d of %d, running time = %ds'%(k+1, model_number, t2-t1)
    for pipe in xrange(pipe_number):
        qh = n1.qhist(pipe)
        N = n1.Ns[pipe]
        A = [qh[idx_t(0, ja, M, N)] for ja in xrange(1,N+1)]
        Q = [qh[idx_t(1, jq, M, N)] for jq in xrange(1,N+1)]
        ICs.update({pipe: [A, Q]})
#     if len(P_beforeT)>0: 
#         # Cut the last number since the begining and the end are overlaped
#         P_beforeT = np.concatenate((P_beforeT[:-1], n1.pressureTimeSeries(2,n1.Ns[2]-1)), axis = 0)  #One grid before the last one 
#         P_afterT = np.concatenate((P_afterT[:-1], n1.pressureTimeSeries(4,0)), axis = 0)
#     else:
#         P_beforeT = n1.pressureTimeSeries(2,n1.Ns[2]-1)
#         P_afterT = n1.pressureTimeSeries(4,0)
#     qh0 = n1.qhist(2)
#     print qh0[100:120]
    n1.reset()
    

In [ ]:
#----------------------------------------------------------------------------
# Calculate Pressure Head Difference 
#----------------------------------------------------------------------------
In = [2]    # In pipe
Out = [3,6] # Out1, Out2
#######################################################################

P0 = n1.pressureTimeSeries(In[0],n1.Ns[In[0]]-1) 
P1 = n1.pressureTimeSeries(Out[0],0)
P2 = n1.pressureTimeSeries(Out[1],0)
t = linspace(0,n1.T, n1.M+1)
# Pressure Head difference dP1 = P0 - P1
dP1 = [P1[i]-P0[i] for i in xrange(len(P0))]
dP2 = [P2[i]-P0[i] for i in xrange(len(P0))]
print "dP1 finally =",dP1[-1]
print "dP2 finally =",dP2[-1]
fig = plt.figure(figsize= (10,5))
plot(t,dP1, label = 'P1-P0')
plot(t,dP2, label = 'P2-P0')
legend()
xlabel("time")
ylabel("Pressue Head (m)")
plt.ylim(-2.5,2.5)

#----------------------------------------------------------------------------
# Calculate Q1, Q2 distribution
#----------------------------------------------------------------------------
# get pipe (A,Q)data
qh0 = n1.qhist(In[0])
qh1 = n1.qhist(Out[0])
qh2 = n1.qhist(Out[1])
N = n1.Ns[1]
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)
N1 = n1.Ns[Out[0]]
N2 = n1.Ns[Out[1]]
Qext1 = [qh1[idx_t(1,N1,it,N1)] for it in xrange(n1.M+1)]
Qext2 = [qh2[idx_t(1,N2,it,N2)] for it in xrange(n1.M+1)]
print "Q1 finally =",Qext1[-1], "Percentage =", Qext1[-1]/q0
print "Q2 finally =",Qext2[-1],"Percentage =", Qext2[-1]/q0
fig = plt.figure(figsize= (10,5))
plot(t,Qext1, label = 'Pipe 1 outflow')
plot(t,Qext2, label = 'Pipe 2 outflow')
legend()
xlabel("time")
ylabel("flux(m^3/s)")
plt.ylim(0, 2.5)


Px = n1.pressureTimeSeries(5,n1.Ns[5]-1) 
print "Pressure Head in Pipe 0 is: ",Px[-1], "m"
print 

In [ ]:
# Display the inflow and outflow for each pipe (m^3)  (ALl PIPES IN MODEL)


# Get Q of two ends of each pipe
Q = []
Mstep=1
#pipe_interest=[0]
pipe_interest=range(0,3)   
'''Here I redefine the pipe_interest vector'''

for i in range(len(pipe_interest)):
    Q_pipefix = [] 
    Q_start=[]
    Q_end=[]
    for m in xrange(0,n1.M,Mstep) :
        j = pipe_interest[i]
        N = n1.Ns[j]
        qh = n1.qhist(j)
        #Q_start.append(qh[idx_t(1,1,m,N)])   # Q1 (since qhist include ghost cell, so 0 and N+1 is the value of ghost cell)
        #Q_end.append(qh[idx_t(1,N,m,N)])     # QN
        Q_start.append(qh[idx_t(0,0,m,N)])   # Q1 (since qhist include ghost cell, so 0 and N+1 is the value of ghost cell)
        Q_end.append(qh[idx_t(1,N+1,m,N)])     # QN
    Q_pipefix.append(Q_start)
    Q_pipefix.append(Q_end)
    Q.append(Q_pipefix)
xt = np.linspace(0, n1.M*dt, n1.M/Mstep)
endorstart=['start','end']
for i in range(len(pipe_interest)):
    plt.figure(figsize= (10,4))                # same pipe on same figure
    for j in range(0,2):  
        #figsize = (15,5)                        # plot on same figure           
        #plt.figure(figsize= (15,5))             # plot on different figures
        plot(xt,Q[i][j], lw = 1,label = 'pipe %d, %s'%(pipe_interest[i],endorstart[j]))
        legend(bbox_to_anchor=(0.7, 0.3), loc=2, borderaxespad=0.)   #legend will be placed out of box and cannot be shown
        #legend()
        xlabel('time (s)')
        ylabel('flux (m^3/s)')
        title('Flux in pipe %d'%pipe_interest[i])
        plt.ylim(0,0.3)

# calculate inflow and outflow, i means pipe number, j means start or end, time means time in vector  
delta_t=Mstep*dt
Q_total=[]
for i in range(len(pipe_interest)):
    Q_total_pipefix=[]
    for j in range(0,2):
        Q_sum=0
        for time in range(len(Q[i][j])):
            Q_sum += Q[i][j][time]*delta_t
        Q_total_pipefix.append(Q_sum)
        print 'Pipe %d during %d s have total flow: %.3f m^3 (%s node)'%(pipe_interest[i],n1.T,Q_sum,endorstart[j])
    V_dif = Q_total_pipefix[0]- Q_total_pipefix[1]
    print V_dif
    Q_total.append(Q_total_pipefix)


# Display mass balance for each junction

#******************************************************************************************
#The following three list must corresponds to each other 
JunNode=[1]
Pipejun_in=[[0]]
Pipejun_out=[[1,2]]
#******************************************************************************************


assert (len(Pipejun_in)==len(Pipejun_out)),"Different dimensions of Pipejun_in and Pipejun_out!"

# Create x axis
xt = np.linspace(0, n1.M*dt, n1.M/Mstep)

# Q[i][j][k]   i means pipe number, j means start or end, k means time in vector 
for i in xrange(len(Pipejun_in)):
    Qjun_dif = []
    print len(Q[0])
    for k in xrange(len(Q[0][1])):
        Qinflow = 0
        Qoutflow = 0
        for j1 in Pipejun_in[i]:
            Qinflow += Q[j1][1][k]
        for j2 in Pipejun_out[i]:
            Qoutflow += Q[j2][0][k]
        Qjun_dif.append(Qinflow-Qoutflow)
    plt.figure(figsize= (10,4)) 
    plot(xt,Qjun_dif)   
    xlabel('t(s)')
    ylabel('Flux difference (m^3/s)')
    title('Junction %d Mass Conservation (Qin-Qout)'%JunNode[i])


In [ ]:
# Overlap
#*********************************************************************************
pipe_interest=[14,3,4]
t_delta=20*dt              # unit: s
t_start=690              # unit:s
t_end= n1.T               # unit: s
#*********************************************************************************

# create x axis
x_interest=[]
pipe_length=0
for j in pipe_interest: 
    x = np.linspace(0,n1.Ls[j],n1.Ns[j])
    x_interest.append(x)

#create initial figure
from matplotlib import animation
fig = plt.figure(figsize= (10,5))
plt.xlim(0,x_interest[-1][-1])
plt.ylim(-0.05, 0.6)
lines = [plt.plot([], [],label='pipe {}'.format(pipe_interest[i]))[0] for i in range(len(pipe_interest))] # number of lines plot on the figure
plt.xlabel('x (m)')        
plt.ylabel('Pressure Head (m)')
plt.title('Pressure Head in pipe %s'%str(pipe_interest)) 
# initialization function: plot the background of each frame

try:
    frac
except NameError:
    frac = 1.
    
for pipe in pipe_interest:
    p0 = PyPipe_ps(n1.Ns[pipe], n1.Ds[pipe],n1.Ls[pipe], M, n1.a[0])
    A0 = n1.Ds[pipe]*n1.Ds[pipe]/4*3.14
    H_full = p0.Eta(A0,True)/9.81/A0 
   
    x = np.linspace(0,n1.Ls[pipe],n1.Ns[pipe])
    plt.plot(x,[H_full]*len(x),'g--',label= 'pipe %d full'%(pipe))
  
    #A_open = p0.AofH(frac*n1.Ds[pipe], False)                                 # farc defined in the running cell
    #H_open= p0.Eta(A_open,False)/9.81/A_open
    #print frac*n1.Ds[pipe],A_open, H_open
    #plt.plot(x,[H_open]*len(x),'r--',label = "pipe %d opening"%(pipe))
legend(bbox_to_anchor=(0.7, 1), loc=2, borderaxespad=0.)




def init():
    for line in lines:
        line.set_data([], [])
    return lines

Mi_draw=int(t_delta/dt)   # difne how many steps to skip
M_start=int(t_start/dt)
M_total=int(t_end/dt)

def animate(index): 
    for i,line in enumerate(lines):             
        j=pipe_interest[i]
        Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw)  
        #this returns H as a function of x in pipe j at time step m
        line.set_data(x_interest[i],Hx)
        plt.xlabel('x (m), t=%.4fs'%((M_start+index*Mi_draw)*dt))
        #line.set_label('pipe %d t=%.2f s'%(j,(dt*(M_start+index*Mi_draw))))
                      
    return lines         
    #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

step = int((M_total-M_start)/Mi_draw)
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=step, interval=3, blit=True)
plt.legend(bbox_to_anchor=(0.85, 1), loc=2, borderaxespad=0.)
plt.show()
"""Save My Result"""
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/Karney_Improvement/Anna+B_1.11_6_open0.8/D2=0.2/%s %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))
# anim.save('/home/xin/pipes/examples/output_data/MyTtest/Two_end/T-normal/all_branch_flow=0.4,combiningflow_all%s.mp4'%str(pipe_interest))
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/surprisingT/beta=0.1,k=0.4%s.mp4'%str(pipe_interest))
# anim.save('/home/xin/pipes/examples/output_data/MyTtest/Two_end/T-normal/H/network_P%s.mp4'%str(pipe_interest))


#anim = animation.FuncAnimation(fig, animate, init_func=init,frames=3, interval=20, blit=True)
#anim.save("basic_animation.mp4")
#plt.show()
    # time.sleep(0.01)
    # bbox_to_anchor: the bbox that the legend will be anchored, 1.05 means 1.05 times of figure length
    # borderaxespad: the pad between the axes and legend border
# Display the inflow and outflow for each pipe (m^3)  (ALl PIPES IN MODEL)

In [ ]:
### Continuous
from matplotlib import animation
#*********************************************************************************
pipe_interest=[2,3,4]
t_delta=20*dt              # unit: s
t_start=100               # unit:s
t_end=n1.T               # unit: s
#*********************************************************************************

# animation already imported above
plt.ion()
plt.show()

# create x axis
x_interest=[]
pipe_length=0
for j in pipe_interest: 
    x = np.linspace(pipe_length,pipe_length+n1.Ls[j],n1.Ns[j])
    x_interest.append(x)
    pipe_length=pipe_length+n1.Ls[j]
x_interest_combine = np.concatenate(x_interest,axis =0)


#create initial figure
fig = plt.figure(figsize= (10,5))
plt.xlim(0,x_interest[-1][-1]+1)
plt.ylim(0, 0.2)
lines = [plt.plot([], [])[0] for i in range(len(pipe_interest))] # number of lines plot on the figure
plt.xlabel('x (m)')        
plt.ylabel('Pressure Head (m)')
plt.title('Pressure Head in pipe %s'%str(pipe_interest))

pipe_length = 0
for pipe in pipe_interest:
    p0 = PyPipe_ps(n1.Ns[pipe], n1.Ds[pipe],n1.Ls[pipe], M, n1.a[0])
    A0 = n1.Ds[pipe]*n1.Ds[pipe]/4*3.14
    H_full = p0.Eta(A0,True)/9.81/A0 
    if pipe == 0:
        x = np.linspace(pipe_length,pipe_length+n1.Ls[pipe],n1.Ns[pipe])
        plt.plot(x,[H_full]*len(x),'g--',label= 'pipe %d full'%(pipe))
        pipe_length += n1.Ls[pipe]
    else:
        x = np.linspace(pipe_length,pipe_length+n1.Ls[pipe],n1.Ns[pipe])
        plt.plot(x,[H_full]*len(x),'g--',label="pipe %d full"%(pipe))
        A_open = p0.AofH(0.8*n1.Ds[pipe], False)
        H_open= p0.Eta(A_open,False)/9.81/A_open
        print 0.8*n1.Ds[pipe],A_open, H_open
        plt.plot(x,[H_open]*len(x),'r--',label = "pipe %d opening"%(pipe))
legend(bbox_to_anchor=(0.7, 1), loc=2, borderaxespad=0.)
# initialization function: plot the background of each frame



def init():
    for line in lines:
        line.set_data([], [])
    return lines

dt = n1.T/n1.M
Mi_draw=int(t_delta/dt)   # difne how many steps to skip
print Mi_draw, dt
M_start=int(t_start/dt)
M_total=int(t_end/dt)
H_interest=[]

def animate(index):
    H_interest_tfixed = []
    for i,line in enumerate(lines):             
        j=pipe_interest[i]
        Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw)  
        #this returns H as a function of x in pipe j at time step m
        H_interest_tfixed.append(Hx)
    H_interest.append(H_interest_tfixed)
    H_interest_combine = np.concatenate(H_interest[index],axis =0)
    line.set_data(x_interest_combine,H_interest_combine)
    plt.xlabel('x (m), t=%.3fs'%((M_start+index*Mi_draw)*dt))
                      
    return lines         
    #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

step = int((M_total-M_start)/Mi_draw)
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=step, interval=3, blit=False)
plt.show()

<h1> Get Excel Data And Plot <h1>

In [ ]:
######## Read Data From Excel File #####################################3
import pandas
t3 = time.time()
df = pandas.read_excel('/home/xin/pipes/indata/Xindata/August2016Export_Excel.xlsx')
t4 = time.time()
print "Reading Excel Data with time = %d s"%(t4-t3)

In [ ]:
#print the column names
#print df.columns
#get the values for a given column
BeforeT_Avg = df['492334 Chnl 01 Avg'].values    
AfterT_Avg = df['492333 Chnl 01 Avg'].values   
Time = df['TimeStamp'].values
print 'Some problems happen with After T data since the min is 2psi with higher elev.'
###############################################################################
#print values[178494]   ##Trial Function, print 178496 row value, so to get nth row value, should use n-2 index
##################################################################################
##### Find rows we need from excel file #############################
start_row = 173190
end_row = 187241
####################################################################################


BeforeT_Avg_23rd = BeforeT_Avg[int(start_row-2):int(end_row-1)]
AfterT_Avg_23rd = AfterT_Avg[int(start_row-2):int(end_row-1)]
Time_23rd = Time[int(start_row-2):int(end_row-1)]

def getIndex(array, target_string):
    indexes = []
    for i in xrange(len(array)):
        if array[i] == target_string:
            indexes.append(i)
    return indexes

c = 0.7028 # Psi_to_Pressure Head

BeforeT_index = getIndex(BeforeT_Avg_23rd, ' - ')
BeforeT_Avg_23rd = np.delete(BeforeT_Avg_23rd,BeforeT_index)  # BeforeT_Avg_23rd type: np.ndarray
BeforeT_Avg_23rd = [i*c for i in BeforeT_Avg_23rd]
AfterT_index = getIndex(AfterT_Avg_23rd, ' - ')
AfterT_Avg_23rd = np.delete(AfterT_Avg_23rd,AfterT_index)  # AfterT_Avg_23rd type: np.ndarray
AfterT_Avg_23rd = [i*c for i in AfterT_Avg_23rd]

import copy
T1 = copy.deepcopy(Time_23rd)
T2 = copy.deepcopy(Time_23rd)
T1 = np.delete(T1,BeforeT_index)
T2 = np.delete(T2,AfterT_index)

#get a data frame with selected columns
#FORMAT = ['Arm_id', 'DSPName', 'Pincode']
#df_selected = df[FORMAT]



################## Define Start Time String and End Time String Based on Simulation ######################
start_time = '8/23/2016 05:03:05'
end_time = '8/23/2016 05:18:05'
##################################################################################################

dt = n1.T/n1.M * 1000  # in milliseconds
# 'start' is a string in H:M:S format, so is 'end'. 'delta' is an int with unit: ms
def date_range(start, end, delta):    
    from datetime import datetime, timedelta
    FMT = "%m/%d/%Y %H:%M:%S"
    start = datetime.strptime(start,FMT)
    end = datetime.strptime(end,FMT) 
    diff = timedelta(milliseconds = delta)
    times = []
    time = start
    while time <= end:
        times.append(time)  #.time() method to extract hour, minutes, seconds and microseconds from datetime
        time = time + diff
    return times
t = date_range(start_time, end_time,dt)

# Get Pressure From Numerical Result
P_beforeT = n1.pressureTimeSeries(2,n1.Ns[2]-2)  #One grid before the last one 
P_afterT = n1.pressureTimeSeries(4,1)  

# Create Plots
fig = plt.figure(figsize= (12,5))
plt.plot(T1, BeforeT_Avg_23rd,'r--',label='Before T')
plt.plot(T2, AfterT_Avg_23rd,'b--',label='After T')
plt.plot(t, P_beforeT, 'r', label ='Numerical Before T')
plt.plot(t, P_afterT,'b', label ='Numerical After T')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Pressure Head (m)')
plt.title('Field Data')

In [ ]:
m32gal = 264.172   #conversion factor, m^3 to gallons
m2psi = 1.42       #conversion factor, m of pressure head to psi
mdx = min([n1.Ls[i]/n1.Ns[i] for i in range(n1.Nedges)])
#print timing information
print "Simulated time T = %f s" %n1.T
print "Solve time = %f  s" %n1.solve_time
print "dt = %f" %dt
print "inflow volume = %.2f gallons" % ((n1.getTotalVolume()-V0)*m32gal)
print "pressure wave speed is %f m/s"% (max(n1.a))
print "CFL = 1/(dx/dt)*(max wave speed) = %f" % (max(n1.cmax)*dt/mdx)

In [ ]:
# i = 0 means A, i=1 means Q
# j means grid number [0,N+1] including two grid cells
# n means time step
# N means grid number of this pipe
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)
H1 = n1.pressureSpaceSeries(0,int(M))  
H2 = n1.pressureSpaceSeries(1,int(M)) 
H3 = n1.pressureSpaceSeries(2,int(M)) 
print len(H1) # so H0 only contains real grid values
print H1[-1],H2[0],H3[0]

# print the pressure head at the junction
#print '0:',p1.pbar(0.151917,True),"1,2:",p1.pbar(0.162652,False)

# get pipe (A,Q)data
qh1 = n1.qhist(0)
qh2 = n1.qhist(1)
qh3 = n1.qhist(2)
N = n1.Ns[1]

# For pipe 1, print Qin,Ain,Qext, Aext (right end)
M = n1.M/2
Ain1 = qh1[idx_t(0,N,int(M),N)]
Aext1 = qh1[idx_t(0,N+1,int(M),N)]
Qin1 = qh1[idx_t(1,N,int(M),N)]
Qext1= qh1[idx_t(1,N+1,int(M),N)]
print "The cell value of pipe 1:",Ain1,Qin1,Aext1,Qext1,p1.Eta(Ain1,False)/9.81/Ain1

# For pipe 2, print Qin,Ain,Qext, Aext (left end)
Ain2 = qh2[idx_t(0,1,int(M),N)]
Aext2 = qh2[idx_t(0,0,int(M),N)]
Qin2 = qh2[idx_t(1,1,int(M),N)]
Qext2= qh2[idx_t(1,0,int(M),N)]
print "The cell value of pipe 2:",Ain2,Qin2,Aext2,Qext2,p1.Eta(Ain2,False)/9.81/Ain1

# For pipe 3, print Qin,Ain,Qext, Aext (left end)
Ain3 = qh3[idx_t(0,1,int(M),N)]
Aext3 = qh3[idx_t(0,0,int(M),N)]
Qin3 = qh3[idx_t(1,1,int(M),N)]
Qext3= qh3[idx_t(1,0,int(M),N)]
print "The cell value of pipe 3:",Ain3,Qin3,Aext3,Qext3,p1.Eta(Ain3,False)/9.81/Ain1

print len(qh1)



In [ ]:
#*********************************************************************************
pipe_interest=[0,1,2]
t_delta=1              # unit: s
t_start=0              # unit:s
t_end= n1.T               # unit: s
#*********************************************************************************

# create x axis
x_interest=[]
pipe_length=0
for j in pipe_interest: 
    x = np.linspace(0,n1.Ls[j],n1.Ns[j])
    x_interest.append(x)

#create initial figure
from matplotlib import animation
fig = plt.figure(figsize= (10,5))
plt.xlim(0,x_interest[-1][-1]+1)
plt.ylim(-0.05, 0.8)
lines = [plt.plot([], [],label='pipe {}'.format(pipe_interest[i]))[0] for i in range(len(pipe_interest))] # number of lines plot on the figure
plt.xlabel('x (m)')        
plt.ylabel('Pressure Head (m)')
plt.title('Pressure Head in pipe %s'%str(pipe_interest)) 
# initialization function: plot the background of each frame

def init():
    for line in lines:
        line.set_data([], [])
    return lines

Mi_draw=int(t_delta/dt)   # difne how many steps to skip
M_start=int(t_start/dt)
M_total=int(t_end/dt)

def animate(index): 
    for i,line in enumerate(lines):             
        j=pipe_interest[i]
        Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw)  
        #this returns H as a function of x in pipe j at time step m
        line.set_data(x_interest[i],Hx)
        plt.xlabel('x (m), t=%ds'%((M_start+index*Mi_draw)*dt))
        #line.set_label('pipe %d t=%.2f s'%(j,(dt*(M_start+index*Mi_draw))))
                      
    return lines         
    #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

step = int((M_total-M_start)/Mi_draw)
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=step, interval=3, blit=True)
plt.legend(bbox_to_anchor=(0.85, 1), loc=2, borderaxespad=0.)
plt.show()
"""Save My Result"""
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/Karney_Improvement/Anna+B_1.11_6_open0.8/D2=0.2/%s %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))

#anim = animation.FuncAnimation(fig, animate, init_func=init,frames=3, interval=20, blit=True)
#anim.save("basic_animation.mp4")
#plt.show()
    # time.sleep(0.01)
    # bbox_to_anchor: the bbox that the legend will be anchored, 1.05 means 1.05 times of figure length
    # borderaxespad: the pad between the axes and legend border
# Display the inflow and outflow for each pipe (m^3)  (ALl PIPES IN MODEL)




In [ ]:
# Calculated Outflow
#****************************************************************************
t_interval = 1                  # unit: s
pipe_interest = range(0,3)      # Pipes you want to know 
#****************************************************************************

def get_gradient(distance,pressure_series):
    gradient = []
    for k in xrange(len(pressure_series)-1):
        gradi = (pressure_series[k]-pressure_series[k+1])/distance
        #if gradi == 0:                     # 0 is a const to reflect initial gradient
            #return 0
        gradient.append(gradi)
    return gradient

Mi_draw = int(t_interval/dt)
average_all=[]
deviation_all=[]
for i in xrange(len(pipe_interest)):
    pipe_number = pipe_interest[i]
    dx = n1.Ls[pipe_number]/n1.Ns[pipe_number]
    t_dev=[]
    deviation = []
    average = []  
    for m in range(0,n1.M+1,Mi_draw):
        Hx = n1.pressureSpaceSeries(pipe_number,m)
        gradient = get_gradient(dx,Hx[2:-2])           # eliminate first two grids and the last grid
        ave = np.mean(gradient)
        gradient = [gradient[loc] / ave for loc in range(len(gradient))]  # normalize gradient
        deviation.append(np.std(gradient))
        average.append(ave)
        t_dev.append(m*dt)
    average_all.append(average)
    deviation_all.append(deviation)
    



# average_all [pipe][time_step]
# Using the average gradient along the pipe to get [This is the equation used in the steady state, so try it]
U_interest=[]
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

g = 9.81
pipe_interest = range(0,3)
plt.figure(figsize= (12,5)) 
for pipe in pipe_interest:
    mean_gradient = average_all[pipe]
    Q_pipefixed = []
    for time in xrange(len(mean_gradient)):
        #using the equation of pressurized short pipe, assume that the pipe is full
        # Q = miu*A*sqrt(2*g*(H2-H1))
        lambuda = 0.037
        miu = 1/sqrt(1+lambuda*n1.Ls[pipe]/n1.Ds[pipe]) # no obvious local loss, only frictional loss
        delta_head = mean_gradient[time]*(n1.Ns[pipe]-3)/n1.Ns[pipe]*n1.Ls[pipe]
        A = n1.Ds[pipe]**2/4*pi
        if delta_head<0:
            Q=-miu*A*sqrt(abs(2*g*delta_head))
        else:
            Q = miu*A*sqrt(abs(2*g*delta_head))
        Q_pipefixed.append(Q) 
    plt.plot(t_dev,Q_pipefixed,label='pipe%d'%pipe) 
    legend()
    plt.xlabel('t(s)')
    plt.ylabel('Q (m3/s)')
    #plt.title('v=%.1f,Pipe %s Outflow (from me)'%(v,str(pipe_interest)))
    #savefig('/home/xin/pipes/examples/output_data/EpipeFig/Outflow/Pipe %s Outflow (from me).png'%str(pipe_interest), bbox_inches='tight')

    
# Compare with Lieb's results from equation [just get Q_end from each pipe]
Q=[]
for i in range(len(pipe_interest)):
    Q_pipefix = [] 
    Q_end=[]
    for m in xrange(0,n1.M,Mi_draw) :
        j = pipe_interest[i]
        N = n1.Ns[j]
        qh = n1.qhist(j)
        Q_end.append(qh[idx_t(1,N-1,m,N)])     # QN
    Q_pipefix.append(Q_end)
    Q.append(Q_pipefix)
xt = np.linspace(0, n1.M*dt, n1.M/Mi_draw)
plt.figure(figsize= (12,5))             # plot on different figures
for i in range(len(pipe_interest)):   
        print len(xt),len(Q[i][0])
        plot(xt,Q[i][0], lw = 1,label = 'pipe %d'%(pipe_interest[i]))
        #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)   #legend will be placed out of box and cannot be shown
        legend()
        xlabel('time (s)')
        ylabel('Q (m3/s)')
        #plt.title('v=%.1f,Pipe %s Outflow (from me)'%(v,str(pipe_interest)))
#savefig('/home/xin/pipes/examples/output_data/EpipeFig/Outflow/Pipe %s Outflow (from Libe).png'%str(pipe_interest), bbox_inches='tight')

In [ ]:
# Get inflow to the system and outflow from the system
#****************************************************************************
t_interval = 3.5*dt                  # unit: s
pipe_interest = range(0,3)      # Pipes you want to know 
#****************************************************************************

def get_gradient(distance,pressure_series):
    gradient = []
    for k in xrange(len(pressure_series)-1):
        gradi = (pressure_series[k]-pressure_series[k+1])/distance
        #if gradi == 0:                     # 0 is a const to reflect initial gradient
            #return 0
        gradient.append(gradi)
    return gradient

Mi_draw = int(t_interval/dt)

Q=[]
for i in range(len(pipe_interest)):
    Q_pipefix = [] 
    Q_end=[]
    for m in xrange(0,n1.M,Mi_draw) :
        j = pipe_interest[i]
        N = n1.Ns[j]
        qh = n1.qhist(j)
        if j == 0:
            Q_end.append(qh[idx_t(1,0,m,N)])
        else:
            Q_end.append(qh[idx_t(1,N-1,m,N)])     # QN
    Q_pipefix.append(Q_end)
    Q.append(Q_pipefix)
xt = np.linspace(0, n1.M*dt, n1.M/Mi_draw+1)
plt.figure(figsize= (12,5))             # plot on different figures
for i in range(len(pipe_interest)): 
        print len(xt),len(Q[i][0])
        plot(xt,Q[i][0], lw = 1,label = 'pipe %d'%(pipe_interest[i]))
        legend(bbox_to_anchor=(0.9, 1), loc=2, borderaxespad=0.)   #legend will be placed out of box and cannot be shown
        #legend()
        plt.ylim(0, 0.3)
        xlabel('time (s)')
        ylabel('Q (m3/s)')
        #plt.title('v=%.1f,Pipe %s Outflow (from me)'%(v,str(pipe_interest)))
#savefig('/home/xin/pipes/examples/output_data/MyTtest/inflowq=0.25,v=1.27/Pipe %s Outflow (from Libe).png'%str(pipe_interest), bbox_inches='tight')

In [ ]:
# Water Volume changes in pipe with time

#*****************************************************************************
pipe_interest1=range(0,3)
#*****************************************************************************

print "The volume in each pipe is",(n1.Ds[0]**2*pi/4)*n1.Ls[0], "m^3"

# Get A of two ends of each pipe
A = [] # all time, all pipes, all A values in grids
M_total=n1.M+1
Mi_draw=100

for m in xrange(0,M_total,Mi_draw) :
    A_tfixed=[] # certain time, all pipes, all A values in grids
    for i in xrange(len(pipe_interest1)):
        j = pipe_interest1[i]
        N = n1.Ns[j]
        qh = n1.qhist(j) 
        Atemp=[]  #certain time, certain pipe, all A values in grids
        for k in xrange(1,N+1):
            Ak=qh[idx_t(0,k,m,N)]      
            Atemp.append(Ak)
        A_tfixed.append(Atemp)     
    A.append(A_tfixed)
        
#calculate total volumn, i means time, j means pipe number, distance means value of grids  
V_total=[]
for i in xrange(0,M_total,Mi_draw):
    V_total_tfix=[]
    for j in xrange(len(pipe_interest1)):
        V_sum=0
        pipe_number=pipe_interest1[j]
        delta_x=n1.Ls[pipe_number]/n1.Ns[pipe_number]
        for distance in xrange(len(A[int(i/Mi_draw)][j])):
            V_sum += A[int(i/Mi_draw)][j][distance]*delta_x
        V_total_tfix.append(V_sum)
        #print 'Pipe %d during %d s have total flow: %.3f m^3 '%(pipe_number,i*dt,V_sum)
    #print 'The total volumn at t=%d s are                      %.3f m^3'%(i*dt,sum(V_total_tfix))
    V_total.append(V_total_tfix)

xt1 = np.linspace(0, M_total*dt, M_total/Mi_draw)
plt.figure(figsize= (11,5))
title('Water Volume in Pipe ~ time')
xlabel('t(s)')
ylabel('V(m^3)')
for j in xrange(len(pipe_interest1)):
    pipe_number=pipe_interest1[j]
    V_total_pipefix=[]
    for i in xrange(1,M_total,Mi_draw):
        V_total_pipefix.append(V_total[int(i/Mi_draw)][j])
    plot(xt1,V_total_pipefix,label = 'pipe %d'%pipe_number)
    plt.ylim(0,20)
    legend(bbox_to_anchor=(0.9, 1), loc=2, borderaxespad=0.)



In [ ]:
#show time series at a sensor in the middle of each pipe in the network
# import a nice colormap
from matplotlib import cm
import matplotlib.colors as colors
cNorm  = colors.Normalize(vmin=0, vmax=Np+1)
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=cm.get_cmap('hsv') )
fig = figure(figsize= (15,5))
t = linspace(0,T,M+1)
interesting = arange(0,Np) #pipes you are interested in (currently all)
interesting1 = array([0,1,2,4,6])
for i in interesting1: 
    #look at pressure time series in i^th pipe, pbar(A(X*, t)), where X* = L/2 for pipe length L, and t =0, dt, 2*dt,...dt*M
    P1 = n1.pressureTimeSeries(i,n1.Ns[i]/2-1) 
    plot(t,P1,color =scalarMap.to_rgba(i), label = "pipe %d" %i) 
legend(ncol=7, loc = (0,1))
ax = gca()
ax.set_xlabel('t (s)')
#xlabel('t(s)')
#ylabel('H(m)')
ax.set_ylabel('H (m)')

In [ ]:
#show pressure time series at different locations along the length of a single pipe
i = 1  #pipe number you want to look at
fig = figure(figsize= (15,5))
#new color map
xs = np.arange(1,n1.Ns[i]+1,4)           # all the grids, increment is 2
print xs
print len(xs)
cNorm  = colors.Normalize(vmin=0, vmax=xs[-1]+10)
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=cm.get_cmap('ocean') )
t = np.linspace(0,n1.T,M+1)
for J in range(0,len(xs),3):
    cval = scalarMap.to_rgba(xs[J])
    dx = n1.Ls[i]/float(n1.Ns[i])
    P2 = n1.pressureTimeSeries(i,J)
    q=plot(t,P2,color =cval,label="x = %.1f"%(dx*xs[J]))
ax = gca()
ax.set_title('Pipe %d'%i)
ax.set_xlabel('t (s)')
ax.set_ylabel(r'$\bar{p}$ (m)')
#legend()
#legend(loc = 'upper left')
#savefig("../sensors_pipe%d.eps"%i, format='eps')

In [ ]:
#show how pressure profile varies in space at the end of the run 
i = 5  # pipe you want to look at
fig = figure(figsize= (10,5))
#m = M  #time slice you want to look at, the simulation only has M steps, so m here refers to the end of the simulation
for m in range(0,n1.M+1,6000):
    Px = n1.pressureSpaceSeries(i,m)
    x = linspace(0,n1.Ls[i],n1.Ns[i])
    plot(x,Px,label = 't=%.2f s'%(dt*m))
    # title('Pipe %d at time %.2f s'%(i,dt*m))
    legend()
    draw()
    xlabel('x')
    ylabel(r'$\bar{p}(m)$')
    fig = figure(figsize= (10,5))


In [ ]:
#plot network layout
(xs,ys,conns,ls) = getBasicConnectivity(fi)
Np = shape(conns)[0]
plotNetworkLayout(xs,ys,conns,ls,Np)

In [ ]:
# The following codes are only used to try python functions, the purpose is about learning python
plt.plot([1,2,3,4], [1,4,9,16], 'ro')
plt.axis([0, 6, 0, 20])
plt.show()


In [ ]:
help(gca)

In [ ]:

1*np.ones(2)

In [ ]:
# Continuous Pressure Head Display
from __future__ import division
import sys
sys.path.append("..")    
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
#%pylab inline
%pylab notebook
from writeit import *
import pickle
import time

#*********************************************************************************
pipe_interest=[2,3,9]
t_delta=1              # unit: s
t_start=0               # unit:s
t_end=3600               # unit: s
#*********************************************************************************

# create x axis
x_interest=[]
pipe_length=0
for j in pipe_interest: 
    x = np.linspace(0,n1.Ls[j],n1.Ns[j])
    x_interest.append(x)

#create initial figure
from matplotlib import animation
fig = plt.figure(figsize= (10,5))
plt.xlim(0,x_interest[-1][-1]+1)
plt.ylim(-0.1, 400)
lines = [plt.plot([], [],label='pipe {}'.format(pipe_interest[i]))[0] for i in range(len(pipe_interest))] # number of lines plot on the figure
plt.xlabel('x (m)')        
plt.ylabel('Pressure Head (m)')
plt.title('Pressure Head in pipe %s'%str(pipe_interest)) 
# initialization function: plot the background of each frame

def init():
    for line in lines:
        line.set_data([], [])
    return lines

Mi_draw=int(t_delta/dt)   # difne how many steps to skip
M_start=int(t_start/dt)
M_total=int(t_end/dt)

def animate(index): 
    for i,line in enumerate(lines):             
        j=pipe_interest[i]
        Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw)  
        #this returns H as a function of x in pipe j at time step m
        line.set_data(x_interest[i],Hx)
        plt.xlabel('x (m), t=%ds'%((M_start+index*Mi_draw)*dt))
        #line.set_label('pipe %d t=%.2f s'%(j,(dt*(M_start+index*Mi_draw))))
                      
    return lines         
    #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

step = int((M_total-M_start)/Mi_draw)
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=step, interval=10, blit=True)
plt.legend(bbox_to_anchor=(0.95, 1), loc=2, borderaxespad=0.)
plt.show()
#anim.save('/home/xin/pipes/examples/output_data/T_junction_detection/pipe %s %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))